## Mapa

In [5]:
import pandas as pd
import folium

# Cargar el dataset desde el archivo JSON
file_path = "dataset/transacciones_actualizadas.json"  # Corrige el nombre del archivo
df = pd.read_json(file_path)  # Añadido lines=True si el JSON tiene múltiples objetos

# Verificar si el dataset tiene datos
if df.empty:
    print("El dataset está vacío. Verifica el archivo JSON.")
else:
    # Crear un mapa centrado en las coordenadas promedio del dataset
    centro_latitud = df["latitud"].mean()
    centro_longitud = df["longitud"].mean()
    mapa = folium.Map(location=[centro_latitud, centro_longitud], zoom_start=12)

    # Añadir marcadores para cada transacción en el dataset
    for _, fila in df.iterrows():
        # Verificar que las coordenadas sean válidas
        if pd.notnull(fila["latitud"]) and pd.notnull(fila["longitud"]):
            popup_info = f"""
            <b>ID Transacción:</b> {fila['id_transaccion']}<br>
            <b>Monto:</b> {fila['monto']}<br>
            <b>Ordenante:</b> {fila['ordenante_nombre_completo']}<br>
            <b>Beneficiario:</b> {fila['beneficiario_nombre_completo']}
            """

            folium.Marker(
                location=[fila["latitud"], fila["longitud"]],
                popup=folium.Popup(popup_info, max_width=300),
                icon=folium.Icon(color="blue", icon="info-sign")
            ).add_to(mapa)

    # Guardar el mapa en un archivo HTML
    output_path = "mapa_transacciones.html"
    mapa.save(output_path)
    print(f"Mapa interactivo guardado en: {output_path}")


ValueError: Expected object or value

In [6]:
import pandas as pd
import folium

# Ruta al archivo JSON
file_path = "dataset/transacciones_actualizadas.json"

# Cargar el JSON como lista
try:
    df = pd.read_json(file_path)  # No usar `lines=True`
    print("Datos cargados exitosamente.")
except ValueError as e:
    print(f"Error al cargar el archivo JSON: {e}")
    raise

# Verificar si el dataset tiene datos
if df.empty:
    raise ValueError("El archivo JSON no contiene datos o está vacío.")

# Crear un mapa centrado en las coordenadas promedio del dataset
centro_latitud = df["latitud"].mean()
centro_longitud = df["longitud"].mean()
mapa = folium.Map(location=[centro_latitud, centro_longitud], zoom_start=12)

# Añadir marcadores para cada transacción en el dataset
for _, fila in df.iterrows():
    popup_info = f"<b>ID Transacción:</b> {fila['id_transaccion']}<br>" \
                 f"<b>Monto:</b> {fila['monto']}<br>" \
                 f"<b>Ordenante:</b> {fila['ordenante_nombre_completo']}<br>" \
                 f"<b>Beneficiario:</b> {fila['beneficiario_nombre_completo']}"

    folium.Marker(
        location=[fila["latitud"], fila["longitud"]],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
output_path = "mapa_transacciones.html"
mapa.save(output_path)
print(f"Mapa interactivo guardado en: {output_path}")


Datos cargados exitosamente.
Mapa interactivo guardado en: mapa_transacciones.html


In [37]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Cargar el dataset desde un archivo JSON
file_path = r"dataset\transacciones_actualizadas.json"

# Cargar y verificar las columnas del dataset
df = pd.read_json(file_path)
print("Columnas disponibles en el dataset:")
print(df.columns)

# Definir las columnas relevantes
variables_interes = [
    "ordenante_cedula",  # Identificador único del usuario
    "monto",
    "fecha_hora",
    "local_nombre",
    "latitud",
    "longitud"
]

# Asegurarse de que las columnas relevantes estén en el dataset
for columna in variables_interes:
    if columna not in df.columns:
        raise KeyError(f"La columna '{columna}' no se encuentra en el dataset.")

# Filtrar las columnas relevantes
df = df[variables_interes]

Columnas disponibles en el dataset:
Index(['id_transaccion', 'fecha_hora', 'monto', 'latitud', 'longitud',
       'dispositivo_id', 'ordenante_nombre_completo',
       'ordenante_numero_cuenta', 'ordenante_cedula',
       'ordenante_institucion_financiera', 'ordenante_codigo_institucion',
       'beneficiario_nombre_completo', 'beneficiario_numero_cuenta',
       'beneficiario_cedula', 'beneficiario_institucion_financiera',
       'beneficiario_codigo_institucion', 'direccion_ip', 'sistema_operativo',
       'categoria', 'motivo', 'local_nombre'],
      dtype='object')


## Limpieza del dataset

In [38]:
# Reemplazar valores nulos en la columna 'local_nombre' por "Desconocido"
df["local_nombre"] = df["local_nombre"].fillna("Desconocido")

# Extraer la hora y el día de la semana de la columna 'fecha_hora'
df["hora"] = pd.to_datetime(df["fecha_hora"]).dt.hour
df["dia_semana"] = pd.to_datetime(df["fecha_hora"]).dt.weekday

# Codificar la variable 'local_nombre' con LabelEncoder
label_encoder = LabelEncoder()
df["local_nombre_encoded"] = label_encoder.fit_transform(df["local_nombre"])

# Normalizar las variables numéricas (monto, latitud, longitud, hora, día de la semana)
scaler = MinMaxScaler()
df[["monto", "latitud", "longitud", "hora", "dia_semana"]] = scaler.fit_transform(
    df[["monto", "latitud", "longitud", "hora", "dia_semana"]]
)

# Eliminar la columna original 'fecha_hora' ya que ahora está representada como 'hora' y 'dia_semana'
df = df.drop(columns=["fecha_hora"])

# Mostrar las primeras filas del dataset transformado
print("Dataset transformado:")
print(df.head())


Dataset transformado:
   ordenante_cedula     monto      local_nombre   latitud  longitud      hora  \
0        3597400581  0.977787       Desconocido  0.000018  0.999916  0.086957   
1        3597400581  0.769383       Desconocido  0.000004  0.999988  0.000000   
2        3597400581  0.401947       Desconocido  0.000007  0.999909  0.652174   
3        3597400581  0.602442  Bogati Heladeria  0.999850  0.002208  0.304348   
4        3597400581  0.958983       Magic Retro  0.998453  0.002660  0.782609   

   dia_semana  local_nombre_encoded  
0    1.000000                     3  
1    0.333333                     3  
2    0.500000                     3  
3    0.500000                     0  
4    0.500000                     7  


In [39]:
# Guardar el dataset transformado en un nuevo archivo JSON
output_path = "dataset/transacciones_transformadas.json"
df.to_json(output_path, orient="records", lines=True)
print(f"Dataset transformado guardado en: {output_path}")

Dataset transformado guardado en: dataset/transacciones_transformadas.json


## Dividir el dataset para entrenamiento y para testeo

In [40]:
from sklearn.model_selection import train_test_split
# Cargar el dataset limpio
file_path = "dataset/transacciones_transformadas.json"
df = pd.read_json(file_path, lines=True)

# Dividir los datos por usuario (ordenante_cedula)
usuarios = df["ordenante_cedula"].unique()

# Crear un conjunto de usuarios para entrenamiento y testeo
usuarios_train, usuarios_test = train_test_split(
    usuarios, test_size=0.2, random_state=42
)

# Separar el dataset en base a los usuarios seleccionados
df_train = df[df["ordenante_cedula"].isin(usuarios_train)]
df_test = df[df["ordenante_cedula"].isin(usuarios_test)]

# Guardar los datasets de entrenamiento y testeo en archivos separados
train_path = "dataset/transacciones_train.json"
test_path = "dataset/transacciones_test.json"

df_train.to_json(train_path, orient="records", lines=True)
df_test.to_json(test_path, orient="records", lines=True)

print(f"Dataset de entrenamiento guardado en: {train_path}")
print(f"Dataset de testeo guardado en: {test_path}")
print("Dataset de entrenamiento:")
print(df_train.head())

Dataset de entrenamiento guardado en: dataset/transacciones_train.json
Dataset de testeo guardado en: dataset/transacciones_test.json
Dataset de entrenamiento:
     ordenante_cedula     monto      local_nombre   latitud  longitud  \
100        4557997274  0.627774  Bogati Heladeria  0.999850  0.002208   
101        4557997274  0.055855       Desconocido  0.000024  0.999990   
102        4557997274  0.184097       Desconocido  0.000008  0.999982   
103        4557997274  0.270921       Desconocido  0.000007  0.999911   
104        4557997274  0.191382       Desconocido  0.000012  0.999970   

         hora  dia_semana  local_nombre_encoded  
100  0.434783    0.166667                     0  
101  0.043478    0.166667                     3  
102  0.391304    0.000000                     3  
103  0.913043    0.833333                     3  
104  0.173913    0.000000                     3  


## Creacion de Modelo KNN

In [34]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

# 1. Cargar el dataset
file_path = "dataset/transacciones_train.json"
df = pd.read_json(file_path, lines=True)

# 2. Definir los pesos para las variables
pesos = {
    "latitud": 0.25,
    "longitud": 0.25,
    "hora": 0.25,
    "dia_semana": 0.25
}

# 3. Normalizar las variables relevantes
scaler = MinMaxScaler()
variables_a_normalizar = ["latitud", "longitud", "hora", "dia_semana"]
df_normalizado = df.copy()
df_normalizado[variables_a_normalizar] = scaler.fit_transform(df[variables_a_normalizar])

# Aplicar los pesos después de la normalización
for columna, peso in pesos.items():
    df_normalizado[columna] *= peso

# 4. Características y modelo KNN
X_recomendador = df_normalizado[["latitud", "longitud", "hora", "dia_semana"]]
recomendador = NearestNeighbors(n_neighbors=5, metric='euclidean')
recomendador.fit(X_recomendador)

# 5. Función para recomendar comercios
def recomendar_por_cedula(cedula, cliente):
    # Filtrar el historial del usuario por cédula
    historial_usuario = df[df["ordenante_cedula"] == cedula]
    
    if historial_usuario.empty:
        print(f"No se encontró historial para el usuario con cédula: {cedula}")
        return None

    # Normalizar los datos del cliente usando los mismos parámetros del scaler
    cliente_normalizado = scaler.transform([[cliente[0], cliente[1], cliente[2], cliente[3]]])
    
    # Aplicar los pesos
    cliente_escalado = [
        cliente_normalizado[0][0] * pesos["latitud"],
        cliente_normalizado[0][1] * pesos["longitud"],
        cliente_normalizado[0][2] * pesos["hora"],
        cliente_normalizado[0][3] * pesos["dia_semana"]
    ]

    # Normalizar y pesar el historial del usuario
    historial_usuario_normalizado = historial_usuario.copy()
    historial_usuario_normalizado[variables_a_normalizar] = scaler.transform(historial_usuario[variables_a_normalizar])
    for columna, peso in pesos.items():
        historial_usuario_normalizado[columna] *= peso

    # Reentrenar el modelo KNN con el historial del usuario
    X_usuario = historial_usuario_normalizado[["latitud", "longitud", "hora", "dia_semana"]]
    recomendador.fit(X_usuario)

    # Realizar la predicción
    distancias, indices = recomendador.kneighbors([cliente_escalado])
    comercios_cercanos = historial_usuario.iloc[indices[0]]

    # Retornar resultados desnormalizados para mayor interpretabilidad
    resultados = comercios_cercanos.copy()
    for columna in variables_a_normalizar:
        resultados[columna] = scaler.inverse_transform(
            [[0] * variables_a_normalizar.index(columna) + [comercios_cercanos[columna].iloc[0]] + [0] * (3 - variables_a_normalizar.index(columna))])[0][variables_a_normalizar.index(columna)]

    return resultados[["local_nombre", "latitud", "longitud", "hora", "dia_semana"]]

# 6. Ejemplo de uso
# Datos del cliente (latitud, longitud, hora, día de la semana)
cedula_cliente = 8927295120
cliente_datos = [-4.0006139264, -79.2049413086, 16, 5]  # Valores originales desnormalizados

# Realizar la recomendación
recomendaciones = recomendar_por_cedula(cedula_cliente, cliente_datos)

# Imprimir los resultados
if recomendaciones is not None:
    print("Comercios más cercanos basados en el historial del usuario (desnormalizados):")
    print(recomendaciones)


Comercios más cercanos basados en el historial del usuario (desnormalizados):
             local_nombre   latitud  longitud      hora  dia_semana
406        Chifa Oriental  0.999268  0.001853  0.956522         1.0
405        Chifa Oriental  0.999268  0.001853  0.956522         1.0
409  Manantial restaurant  0.999268  0.001853  0.956522         1.0
403        Chifa Oriental  0.999268  0.001853  0.956522         1.0
407        Chifa Oriental  0.999268  0.001853  0.956522         1.0


c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


## Ejemplo:

In [26]:
# Ubicación del cliente y datos contextuales (latitud, longitud, hora, día de la semana)
cedula_cliente = 8927295120  # Cédula del cliente
cliente_datos = [0.199854, 0.000371,0.956522 , 1.000000]

# Realizar la recomendación
recomendaciones = recomendar_por_cedula(cedula_cliente, cliente_datos)

# Imprimir los resultados
if recomendaciones is not None:
    print("Comercios más cercanos basados en el historial del usuario (normalizados):")
    print(recomendaciones)

Comercios más cercanos basados en el historial del usuario (normalizados):
             local_nombre   latitud  longitud      hora  dia_semana
406        Chifa Oriental  0.199854  0.000371  0.956522    1.000000
409  Manantial restaurant  0.199852  0.000664  0.869565    1.000000
401           Desconocido  0.000002  0.199999  0.652174    1.000000
407        Chifa Oriental  0.199854  0.000371  0.695652    0.833333
426        Chifa Oriental  0.199854  0.000371  0.434783    0.833333


c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
